# Personal Information
Name: **Wout Gerdes**

StudentID: **15018997**

Email: [**wout.gerdes@student.uva.nl**](youremail@student.uva.nl)

Submitted on: **22.03.2024**

# Data Context
**In this section you should introduce the datasources and datasets which you will be working with. Explain where they are from as well as their domain. Give an overview of what the context of the data is. You should not spend more than 1 to 2 paragraphs here as the core information will be in the next section.**

The PaySim dataset is a synthetic dataset generated by PaySim, a mobile money simulation platform designed to mimic the transaction activities in a mobile money system. It was created for research purposes and is freely available for academic use. The dataset simulates a mobile money transfer network, capturing various aspects of financial transactions, such as transfers, cash withdrawals, deposits, and purchases.

The data includes information about the origin and destination of transactions, transaction types, transaction amounts, balances before and after transactions, timestamps, and flags indicating fraudulent transactions. It is a well-studies dataset due to its similarities to "real-world" fraud and amount of data. Also, many other algorithms and projects have been run on this dataset, allowing for ease of comparison in performance between studies.

An extra dataset for peace of mind is deemed unnecessary due to the vast amount of research using the PaySim dataset, including recent studies at the UvA in a similar task.

# Data Description

**Present here the results of your exploratory data analysis. Note that there is no need to have a "story line" - it is more important that you show your understanding of the data and the methods that you will be using in your experiments (i.e. your methodology).**

**As an example, you could show data, label, or group balances, skewness, and basic characterizations of the data. Information about data frequency and distributions as well as results from reduction mechanisms such as PCA could be useful. Furthermore, indicate outliers and how/why you are taking them out of your samples, if you do so.**

**The idea is, that you conduct this analysis to a) understand the data better but b) also to verify the shapes of the distributions and whether they meet the assumptions of the methods that you will attempt to use. Finally, make good use of images, diagrams, and tables to showcase what information you have extracted from your data.**

# Imports
import os
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

### Data Loading

df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

### Analysis 1: 
Observing the dataframe

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Big difference between isFlaggedFraud fraud and isFraud
df['isFlaggedFraud'].value_counts()

In [ ]:
# Checking value counts in fraud column
df['isFraud'].value_counts()

In [ ]:
# As we can see, the dataset is very imbalanced
fraud_counts = df['isFraud'].value_counts()
total_count = fraud_counts.sum()

percentage_0 = (fraud_counts[0] / total_count) * 100
percentage_1 = (fraud_counts[1] / total_count) * 100

# Print the percentages
print(f"Percentage of 'isFraud' = 0: {percentage_0:.2f}%")
print(f"Percentage of 'isFraud' = 1: {percentage_1:.2f}%")

In [ ]:
# Check for null
df.isnull().sum()

In [ ]:
# Check for duplicates
df.duplicated().sum()

In [ ]:
# Quick statistical summary
df.describe().T

In [ ]:
# Type check
df.info()

### Analysis 2:
Visual analysis

In [ ]:
# Show the NUMBER of transactions per type in the dataset 
type = df['type'].value_counts()
transaction = type.index
count = type.values

# drawing piechart
plt.figure(figsize=(8,8))
plt.pie(count, labels=transaction)
plt.legend(loc='upper right')
plt.show()

# Plots the AMOUNT of FRAUD versus notFRAUD per type of transaction
plt.figure(figsize=(10,6))
ax=sns.countplot(x = "type", hue="isFraud", data = df)
plt.title('Types of Transaction nonFraud and Fraud')
for p in ax.patches:
        ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x(), p.get_height()))

In [ ]:
# Plot the AVERAGE amount per transaction for FRAUD and NONFRAUD 
plt.figure(figsize=(10,6))
ax = sns.barplot(x="type", y="amount", hue="isFraud", data=df.groupby(["type", "isFraud"])["amount"].mean().reset_index())
plt.title('Average Amount of Transactions for nonFraud and Fraud by Type')

# Annotate with average transaction amounts
for p in ax.patches:
    ax.annotate('{:.2f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.show()

In [ ]:
# Violin plot showing the distribution of the TRANSACTION AMOUNT for all types and FRAUD/NONFRAUD
# Distribution is very skewed, and needs further analysis
plt.figure(figsize=(10, 6))
sns.violinplot(y="amount", data=df)
plt.title('Distribution of Transaction Amounts')
plt.ylabel('Amount')
plt.show()

In [ ]:
# Violin plots of TRANSACTION AMOUNT per FRAUD or NON FRAUD
# Indicates a different distribution in fraudulent vs non fraudulent transactions, but this should be further investigated based on type
plt.figure(figsize=(10, 6))
sns.violinplot(x="isFraud", y="amount", data=df)
plt.title('Distribution of Transaction Amounts by Fraud Status')
plt.xlabel('isFraud')
plt.ylabel('Amount')
plt.show()

In [ ]:
# Showing violin plots for types and is fraud
# Hard to read figure, will be viewed more in depth later
plt.figure(figsize=(14, 8))
sns.violinplot(x="type", y="amount", hue="isFraud", data=df)
plt.title('Distribution of Transaction Amounts by Type and Fraud Status')
plt.xlabel('Transaction Type')
plt.ylabel('Amount')
plt.legend(title='isFraud', loc='upper right')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Violin plot of FRAUD and NON FRAUD in only TRANSFER type transactions
# Indicates interesting difference in distribution of transaction amount
plt.figure(figsize=(10, 6))

# Filter the DataFrame for transactions of type 'TRANSFER'
transfer_df = df[df['type'] == 'TRANSFER']

# Create the violin plot
sns.violinplot(x="isFraud", y="amount", data=transfer_df)
plt.title('Distribution of Transaction Amounts for TRANSFER Type by Fraud Status')
plt.xlabel('isFraud')
plt.ylabel('Amount')
plt.show()

In [ ]:
# Violin plot of FRAUD and NON FRAUD in only CASH OUT type transactions
# Indicates interesting difference in distribution of transaction amount

plt.figure(figsize=(10, 6))

# Filter the DataFrame for transactions of type 'CASH_OUT'
cash_out_df = df[df['type'] == 'CASH_OUT']

# Create the violin plot
sns.violinplot(x="isFraud", y="amount", data=cash_out_df)
plt.title('Distribution of Transaction Amounts for CASH_OUT Type by Fraud Status')
plt.xlabel('isFraud')
plt.ylabel('Amount')
plt.show()

### Analysis 3:
Only transfer and cash_out type have fraudulent transactions. Therefore, we zoom in on these types

# Create fraud and valid DF for analysis
fraud = df[df["isFraud"] == 1]
valid = df[df["isFraud"] == 0]

In [ ]:
fraud_transfer = fraud[fraud["type"] == "TRANSFER"]
fraud_cashout = fraud[fraud["type"] == "CASH_OUT"]

# checking if the recipient account of a fraudulent transfer was used as a sending account for cashing out 
fraud_transfer.nameDest.isin(fraud_cashout.nameOrig).any()

In [ ]:
#We derive a new feature Transaction Type2 from these features account types "C" (customer) and "M" (merchant), 
#which would be the first character for each value under nameOrig and nameDest.

#We will create a categorical variable with levels "CC" (Customer to Customer), 
#"CM" (Customer to Merchant), "MC" (Merchant to Customer), "MM" (Merchant to Merchant).

data_new = df.copy() # creating copy for security
data_new["Type2"] = np.nan # initializing feature column

# filling feature column
data_new.loc[df.nameOrig.str.contains('C') & df.nameDest.str.contains('C'),"Type2"] = "CC" 
data_new.loc[df.nameOrig.str.contains('C') & df.nameDest.str.contains('M'),"Type2"] = "CM"
data_new.loc[df.nameOrig.str.contains('M') & df.nameDest.str.contains('C'),"Type2"] = "MC"
data_new.loc[df.nameOrig.str.contains('M') & df.nameDest.str.contains('M'),"Type2"] = "MM"

In [ ]:
# Plotting the differences in FRAUD and NON FRAUD datasets next to each other, based on multiple interesting features
plt.figure(figsize=(25, 16))
plt.subplot(2,2,1)
sns.boxplot(x = 'isFraud', y = 'step', data = data_new)
plt.title('step vs Fraud',fontweight="bold", size=20)
plt.subplot(2,2,2)
sns.boxplot(x = 'isFraud', y = 'amount', data = data_new,palette="husl")
plt.title('amount vs Fraud',fontweight="bold", size=20)
plt.subplot(2,2,3)
sns.boxplot(x = 'isFraud', y = 'oldbalanceOrg', data = data_new, palette= 'husl')
plt.title('oldbalanceOrig vs Fraud',fontweight="bold", size=20)
plt.subplot(2,2,4)
sns.boxplot(x = 'isFraud', y = 'oldbalanceDest', data = data_new,palette="bright")
plt.title('oldbalanceDest vs Fraud',fontweight="bold", size=20)

In [ ]:
fraud = data_new[data_new["isFraud"] == 1]
valid = data_new[data_new["isFraud"] == 0]

In [ ]:
print("Fraud transactions by type1: \n",fraud.Type2.value_counts())
print("\n Valid transactions by type1: \n",valid.Type2.value_counts())

In [ ]:
# Checking for discrepancies in balances, both in incoming and outgoing accounts
wrong_orig_bal = sum(df["oldbalanceOrg"] - df["amount"] != df["newbalanceOrig"])
wrong_dest_bal = sum(df["newbalanceDest"] + df["amount"] != df["newbalanceDest"])
print("Percentage of observations with balance errors in the account giving money: ", 100*round(wrong_orig_bal/len(df),2))
print("Percentage of observations with balance errors in the account receiving money: ", 100*round(wrong_dest_bal/len(df),2))

### Analysis 4:
Analysis of transactions in the time dimension

In [ ]:
#Time analysis, plotting FRAUD and NON FRAUD transactions over time

bins = 50

valid.hist(column="step",color="green",bins=bins)
plt.xlabel("1 hour time step")
plt.ylabel("# of transactions")
plt.title("# of valid transactions over time")

fraud.hist(column ="step",color="red",bins=bins)
plt.xlabel("1 hour time step")
plt.ylabel("# of transactions")
plt.title("# of fraud transactions over time")

plt.tight_layout()
plt.show()

In [ ]:
#Dividing the time bins into DAYS and HOURS, allowing analysis of different DAYS of the WEEK for FRAUD vs NON FRAUD
num_days = 7
num_hours = 24
fraud_days = (fraud.step//num_hours)% num_days
fraud_hours = fraud.step % num_hours
valid_days = (valid.step//num_hours) % num_days
valid_hours = valid.step % num_hours

# plotting scatterplot of the days of the week, identifying the fraudulent transactions (red) from the valid transactions (green) 
plt.subplot(1, 2, 1)
fraud_days.hist(bins=num_days,color="red")
plt.title('Fraud transactions by Day')
plt.xlabel('Day of the Week')
plt.ylabel("# of transactions")

plt.subplot(1,2,2)
valid_days.hist(bins=num_days,color="green")
plt.title('Valid transactions by Day')
plt.xlabel('Day of the Week')
plt.ylabel("# of transactions")

plt.tight_layout()
plt.show()

In [ ]:
# Using the division into HOURS to analyse FRAUD vs NON FRAUD transactions per hour of a day
plt.subplot(1, 2, 1)
fraud_hours.hist(bins=num_hours, color="red")
plt.title('Fraud transactions by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel("# of transactions")


plt.subplot(1, 2, 2)
valid_hours.hist(bins=num_hours, color="green")
plt.title('Valid transactions by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel("# of transactions")

plt.tight_layout()
plt.show()